In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix
import numpy as np

train_path = r'/Users/hexue/pyproject/retina_classification/Retinal Fundus Images/train'
test_path = r'/Users/hexue/pyproject/retina_classification/Retinal Fundus Images/test'
val_path = r'/Users/hexue/pyproject/retina_classification/Retinal Fundus Images/val'


# 定义数据预处理和加载
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


train_data = datasets.ImageFolder(train_path, transform=transform)
val_data = datasets.ImageFolder(val_path, transform=transform)
test_data = datasets.ImageFolder(test_path, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)



In [16]:
# 定义预训练的VGG16模型
vgg16_model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

# 冻结VGG16的参数
for param in vgg16_model.parameters():
    param.requires_grad = False

# 修改VGG16的最后一层，输出11个类别
num_classes = 11
vgg16_model.classifier[-1] = nn.Linear(4096, num_classes)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(vgg16_model.parameters(), lr=0.001)


In [25]:
# ... （上面的定义模型、损失函数和优化器等代码）

for epoch in range(num_epochs):
    vgg16_model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = vgg16_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = correct_train / total_train
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader)}, Train Accuracy: {100 * train_accuracy:.2f}%')

    # 验证循环
    vgg16_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    for images, labels in val_loader:
        outputs = vgg16_model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()

    val_accuracy = correct_val / total_val
    print(f'Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {100 * val_accuracy:.2f}%')


Epoch [1/10], Train Loss: 1.078553980892631, Train Accuracy: 58.56%
Validation Loss: 1.2969116398266383, Validation Accuracy: 54.50%
Epoch [2/10], Train Loss: 1.0758613483731154, Train Accuracy: 58.95%
Validation Loss: 1.1748696225030082, Validation Accuracy: 56.35%
Epoch [3/10], Train Loss: 1.0548467787018247, Train Accuracy: 59.68%
Validation Loss: 1.296981815780912, Validation Accuracy: 55.89%


KeyboardInterrupt: 

In [ ]:
# 在测试集上测试模型
vgg16_model.eval()
test_loss = 0.0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = vgg16_model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 输出测试集的损失和混淆矩阵
print(f'Test Loss: {test_loss/len(test_loader)}')
conf_matrix = confusion_matrix(all_labels, all_preds)
print('Confusion Matrix:')
print(conf_matrix)